In [2]:
#-------------------------START-------------------------
# I have learned from LangchainDocs
# Source link: https://docs.langchain.com/oss/python/deepagents/human-in-the-loop

In [3]:
import uuid
from langchain_core.tools import tool
from deepagents import create_deep_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command

# Basic configuration

In [4]:
@tool
def delete_file(path: str) -> str:
    """Delete a file from the filesystem"""
    return f"Deleted {path}"

@tool
def read_file(path: str) -> str:
    """Read a file from the filesystem"""
    return f"Contents of {path}"

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email"""
    return f"Sent email to {to}"

# Checkpointer is REQUIRED fro human-in-the-loop
checkpointer = MemorySaver()

agent = create_deep_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[delete_file, read_file, send_email],
    interrupt_on={
        "delete_file": True,  # Default: approve, edit, reject
        "read_file": False, # No interrupts needed
        "send_email": {"allowed_decisions": ["approve", "reject"]},  # No editing
    },
    checkpointer=checkpointer # Required
)


# Decision types

In [5]:

# interrupt_on = {
#     # Sensitive operations: allow all options
#     "delete_file": {"allowed_decisions": ["approve", "edit", "reject"]},

#     # Moderate risk: approval or rejection only
#     "write_file": {"allowed_decisions": ["approve", "reject"]},

#     # Must approve (no rejection allowed)
#     "critical_operation": {"allowed_decisions": ["approve"]},
# }

# Handle interrupts

In [6]:
# Create config with thread_id for state persistence
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# Invoke the agent
result = agent.invoke({
        "messages": [{
            "role": "user",
            "content" : "Delete the example.txt"
        }]
    },
    config=config)

# Check if execution was interrupted
if result.get("__interrupt__"):

    # Extract interrupt information
    interrupts = result["__interrupt__"][0].value
    action_requests = interrupts["action_requests"]
    review_configs = interrupts["review_configs"]

    # Create a lookup map from tool name to review config
    config_map = {cfg["action_name"]: cfg for cfg in review_configs}

    # Display the pending actions to user
    for action in action_requests:
        review_config = config_map[action["name"]]
        print(f"Tool: {action['name']}")
        print(f"Arguments: {action["args"]}")
        print(f"Allowed decisions: {review_config['allowed_decisions']}")

    # Get user decisions (one per action_request, in order)
    decisions = [
        {"type": "approve"} # User approved the deletion
    ]
    final_result = agent.invoke(
        {"type": "human_decision", "value": decisions},
        config=config  # configni qayta yuborish majburiy
    )
    # Process final result
    print(result["messages"][-1].content)

   

Tool: delete_file
Arguments: {'path': 'example.txt'}
Allowed decisions: ['approve', 'edit', 'reject']



# Multiple tool calls

In [8]:
config = {"configurable":{"thread_id": str(uuid.uuid4())}}

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Delete temp.txt and send an email to admin@example.com"
            }
        ]   
        },
    config=config
)

if result.get("__interrupt__"):
    interrupts = result["__interrupt__"][0].value
    action_requests = interrupts["action_requests"]

    # Two tools need approval

    assert len(action_requests) == 2

    # Provide decisions in the order as action_requests

    decisions = [
        {"type": "approve"}, # First tool: delete_file
        {"type": "reject"} # Second tool: send_email
    ]

    result = agent.invoke(
        Command(resume={"decisions": decisions}),
        config=config
    )

In [9]:
print(result["messages"][-1].content)

I have deleted `temp.txt`. However, I was unable to send an email to `admin@example.com` because the user rejected the tool call.


# Edit tool argements

In [10]:
if result.get("__interrupt__"):
    interrupts = result["__interrupt__"][0].value
    action_request = interrupts["action_requests"][0]

    # Original args from the agent
    print(action_request["args"])  # {"to": "everyone@company.com", ...}

    # User decides to edit the recipient
    decisions = [{
        "type": "edit",
        "edited_action": {
            "name": action_request["name"], # Must include tool name
            "args": {"to": "team@company.com", "subject": "...", "body": "..."}
        }
    }]

    result = agent.invoke(
        Command(resume={"decisions": decisions}),
        config=config
    )

# Subagent interrupts

In [11]:
agent = create_deep_agent(
    tools=[delete_file, read_file],
    interrupt_on={
        "delete_file": True,
        "read_file": False
    },
    subagents=[{
        "name": "file_manager",
        "description": "Manages file operations",
        "system_prompt": "You are a file managment assistant.",
        "tools": [delete_file, read_file],
        "interrupt_on": {
            # Override: require approval for reads this subagent
            "delete_file": True,
            "read_file": True,  # Different from main agent!
        }
    }],
    checkpointer=checkpointer
)